In [ ]:
%load_ex signature

In [5]:
! pip install helpers

In [74]:
%matplotlib inline

import json
import time
 
import requests
import helpers

from tqdm.notebook import tqdm

import pandas as pd

In [23]:
classes = {
    "Organization": "Q43229", #includes companies
    "Name": "Q82799",
    "Artist": "Q483501",
    "Geolocation": "Q2221906",
    "City": "Q515",
    "Capital": "Q5119",
    "Town": "Q3957",
    "Demonym": "Q217438",
    "Product": "Q2424752",
    "Brand": "Q431289",
    "Georegion": "Q82794",
    "Country": "Q6256",
}

In [114]:
url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
subclasses = {}

for label, wikidata_code in tqdm(classes.items()):
    try:
        print(label, wikidata_code)
        query = "SELECT distinct ?class ?classLabel " + \
                "WHERE { ?class wdt:P279|wdt:P279/wdt:P279 wd:" + wikidata_code + ". " + \
                "        SERVICE wikibase:label { bd:serviceParam wikibase:language 'en' } }"
        data = requests.get(url, params={'query': query, 'format': 'json'}).json()
        time.sleep(1)
        subclasses[label] = data
        # print(len(results[label]), 'of results returned.')
    except Exception as e:
        print(str(e))

Organization Q43229
Name Q82799
Artist Q483501
Geolocation Q2221906
City Q515
Capital Q5119
Town Q3957
Demonym Q217438
Product Q2424752
Brand Q431289
Georegion Q82794
Country Q6256



In [ ]:
url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
subclasses = {}

for label, wikidata_code in tqdm(classes.items()):
    try:
        print(label, wikidata_code)
        query = "SELECT distinct ?class ?classLabel " + \
                "WHERE { ?class wdt:P279|wdt:P279/wdt:P279 wd:" + wikidata_code + ". " + \
                "        SERVICE wikibase:label { bd:serviceParam wikibase:language 'en' } }"
        data = requests.get(url, params={'query': query, 'format': 'json'}).json()
        time.sleep(1)
        subclasses[label] = data
        # print(len(results[label]), 'of results returned.')
    except Exception as e:
        print(str(e))

In [175]:
import pickle
pickle.dump(subclasses, open('subclasses.pickle', 'wb'))

In [116]:
for label in classes:
    print(label)
    print(len(subclasses[label]['results']['bindings']), 'subclasses')

Organization
3528 subclasses
Name
308 subclasses
Artist
350 subclasses
Geolocation
1324 subclasses
City
190 subclasses
Capital
15 subclasses
Town
39 subclasses
Demonym
6 subclasses
Product
3838 subclasses
Brand
42 subclasses
Georegion
978 subclasses
Country
51 subclasses


In [134]:
subclasses['Organization']['results']['bindings'][0]

{'class': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q79751'},
 'classLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'oligarchy'}}

In [170]:
subclasses_data = []
unused_classes = []

for class_label in subclasses:
    for result in subclasses[class_label]['results']['bindings']:
        subclass, subclass_label = result['class']['value'], result['classLabel']['value']
        if subclass_label.split('Q')[-1].isnumeric():
            counter += 1
            unused_classes.append(subclass_label)
            continue
        
        subclasses_data.append({ # 'class_uri': subclass,
                                'class_qid': subclass.split('/')[-1],
                                'class_label': class_label,
                                'subclass_label': subclass_label.lower()})

print(len(unused_classes))

1971


In [166]:
subclasses_df = pd.DataFrame(subclasses_data)

In [167]:
subclasses_df.head()

,class_qid,class_label,subclass_label
0,Q79751,Organization,oligarchy
1,Q163740,Organization,nonprofit organization
2,Q193622,Organization,order
3,Q194195,Organization,amusement park
4,Q196776,Organization,certificate authority


In [172]:
pd.DataFrame(unused_classes).to_csv('csv/unused_subclasses.csv', index=False)
pd.DataFrame(unused_classes).head()

,0
0,Q228353
1,Q230717
2,Q396010
3,Q832115
4,Q1020406


In [168]:
! mkdir csv

mkdir: impossible de créer le répertoire « csv »: Le fichier existe


In [169]:
labels_df.to_csv('csv/subclasses_df.csv', index=False)

In [117]:
print('''
Subclasses{1,2}
===============
Organization
3528 subclasses
Name
308 subclasses
Artist
350 subclasses
Geolocation
1324 subclasses
City
190 subclasses
Capital
15 subclasses
Town
39 subclasses
Demonym
6 subclasses
Product
3838 subclasses
Brand
42 subclasses
Georegion
978 subclasses
Country
51 subclasses
''')


Subclasses{1,2}
Organization
3528 subclasses
Name
308 subclasses
Artist
350 subclasses
Geolocation
1324 subclasses
City
190 subclasses
Capital
15 subclasses
Town
39 subclasses
Demonym
6 subclasses
Product
3838 subclasses
Brand
42 subclasses
Georegion
978 subclasses
Country
51 subclasses



In [118]:
print('''
Subclasses*
===========
Organization
18170 subclasses
Name
4133 subclasses
Artist
1250 subclasses
Geolocation
21131 subclasses
City
206 subclasses
Capital
16 subclasses
Town
41 subclasses
Demonym
7 subclasses
Product
125171 subclasses
Brand
90 subclasses
Georegion
11293 subclasses
Country
68 subclasses
''')


Subclasses*
Organization
18170 subclasses
Name
4133 subclasses
Artist
1250 subclasses
Geolocation
21131 subclasses
City
206 subclasses
Capital
16 subclasses
Town
41 subclasses
Demonym
7 subclasses
Product
125171 subclasses
Brand
90 subclasses
Georegion
11293 subclasses
Country
68 subclasses



In [109]:
url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
instances = {}

for label, wikidata_code in tqdm(classes.items()):
    try:
        print(label, wikidata_code)
        query = "SELECT ?entity " + \
                "WHERE { ?entity wdt:P31/wdt:P279* wd:" + wikidata_code + ". " + \
                "        SERVICE wikibase:label { bd:serviceParam wikibase:language 'en' } }"
        data = requests.get(url, params={'query': query, 'format': 'json'}).json()
        instances[label] = data['results']['bindings']
        # print(len(results[label]), 'of results returned.')
    except Exception as e:
        print(str(e))

Organization Q43229
Name Q82799
Artist Q483501
Geolocation Q2221906
City Q515
Capital Q5119
Town Q3957
Demonym Q217438
Product Q2424752
Brand Q431289
Georegion Q82794
Country Q6256



In [111]:
for label in classes:
    print(label)
    print(len(instances[label]), 'results')

Organization
2906668 results
Name
542138 results
Artist
436 results
Geolocation
10584664 results
City
33101 results
Capital
602 results
Town
44858 results
Demonym
620 results
Product
722076 results
Brand
8194 results
Georegion
6164118 results
Country
699 results


In [125]:
instances[label][0]['entity']['value']

'http://www.wikidata.org/entity/Q16'